<a href="https://colab.research.google.com/github/MykhailoMykhailiuk/DS-HW/blob/main/Hw7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357277 sha256=6caf903743ceee4d847cb994124758e9318b68e242556460c1b2a90738a62d11
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
import pandas as pd
from surprise import Dataset, SVD, SVDpp, NMF
from surprise.model_selection import cross_validate, GridSearchCV

In [3]:
data = Dataset.load_builtin('ml-100k', prompt=True)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [21]:
param_grid = {
    'n_factors': [10, 20, 30],
    'n_epochs': [15, 25, 35],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.1]
}

def grid_search(alg, param_grid):
    gs = GridSearchCV(alg, param_grid, measures=['rmse', 'mae'], cv=5)
    gs.fit(data)
    return gs

**SVD**

In [27]:
svd = grid_search(SVD, param_grid)
print(f"Best parameters: {svd.best_params['rmse']}")
best_svd = svd.best_estimator['rmse']
cross_validate(best_svd, data, measures=['RMSE'], cv=5, verbose=True)

Best parameters: {'n_factors': 10, 'n_epochs': 35}
Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9355  0.9324  0.9393  0.9238  0.9407  0.9343  0.0060  
Fit time          1.56    1.26    1.37    1.49    1.38    1.41    0.10    
Test time         0.15    0.12    0.12    0.35    0.19    0.19    0.08    


{'test_rmse': array([0.93550009, 0.93243893, 0.93932329, 0.92379155, 0.94065827]),
 'fit_time': (1.557548999786377,
  1.2597146034240723,
  1.3664956092834473,
  1.489171028137207,
  1.375535249710083),
 'test_time': (0.15372514724731445,
  0.12070274353027344,
  0.12341141700744629,
  0.3478355407714844,
  0.19339776039123535)}

**SVDpp**

In [29]:
svdpp = grid_search(SVDpp, param_grid)
print(f"Best parameters: {svd.best_params['rmse']}")
best_svdpp = svdpp.best_estimator['rmse']
cross_validate(best_svdpp, data, measures=['RMSE'], cv=5, verbose=True)

Best parameters: {'n_factors': 10, 'n_epochs': 35}
Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9250  0.9274  0.9127  0.9187  0.9146  0.9197  0.0057  
Fit time          20.16   20.21   20.12   20.24   19.85   20.11   0.14    
Test time         5.60    5.83    5.98    5.75    6.53    5.94    0.32    


{'test_rmse': array([0.924997  , 0.92736654, 0.91274321, 0.91866312, 0.91462374]),
 'fit_time': (20.16061043739319,
  20.209139108657837,
  20.116336345672607,
  20.240589380264282,
  19.847493171691895),
 'test_time': (5.601269721984863,
  5.8344621658325195,
  5.981809616088867,
  5.75383734703064,
  6.532452344894409)}

**NMF**

In [28]:
param_grid = {
    'n_factors': [10, 20, 30],
    'n_epochs': [15, 25, 35]
}
nmf = grid_search(NMF, param_grid)
print(f"Best parameters: {nmf.best_params['rmse']}")
best_nmf = nmf.best_estimator['rmse']

cross_validate(best_nmf, data, measures=['RMSE'], cv=5, verbose=True)

Best parameters: {'n_factors': 10, 'n_epochs': 35}
Evaluating RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9766  0.9907  0.9697  0.9913  0.9764  0.9809  0.0086  
Fit time          1.36    1.82    1.63    1.49    2.01    1.66    0.23    
Test time         0.10    0.42    0.16    0.16    0.42    0.25    0.14    


{'test_rmse': array([0.97663012, 0.99065234, 0.96968478, 0.99127963, 0.97635826]),
 'fit_time': (1.361069917678833,
  1.8182356357574463,
  1.6293740272521973,
  1.4929006099700928,
  2.0058300495147705),
 'test_time': (0.10020685195922852,
  0.4242892265319824,
  0.16177964210510254,
  0.1614072322845459,
  0.4223361015319824)}

**Висновок**

Найточнішою виявилась SVDpp модель з середнім показником RMSE на 5 розбиттях в 0.91, але вона також виявилась найбільш ресурсо-затратною і показала найдовший час навчання і тестування даних. Порівнявши всі моделі можна сказати, що найбільш оптимальною є SVD модель, при якій RMSE практично рівна до RMSE в SVDpp моделі, але час навчання і тестування значно кращий.